In [1]:

import logging
from telegram import Update, Bot
from telegram.ext import Updater, CommandHandler, MessageHandler, Filters, CallbackContext

# Replace 'YOUR_BOT_TOKEN' with the token you got from the BotFather
bot = Bot(token=API_TOKEN)
updater = Updater(token=API_TOKEN, use_context=True)
dispatcher = updater.dispatcher

# Enable logging
logging.basicConfig(format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
                     level=logging.INFO)


def send_new_channel_messages(update: Update, context: CallbackContext):
    # Replace 'YOUR_CHANNEL_ID' with the ID of the channel from which you want to read messages
    channel_id = 'my life'

    # Get the list of all updates in the channel
    channel_updates = bot.get_chat_updates(channel_id)

    # Get the user ID to send messages
    user_id = update.message.from_user.id

    # Iterate through all the updates and send new messages to the user
    for update in channel_updates:
        if update.message:
            bot.forward_message(chat_id=user_id, from_chat_id=channel_id, message_id=update.message.message_id)


def start(update: Update, context: CallbackContext):
    user_id = update.message.from_user.id
    update.message.reply_text(f"Hello! I will send you new messages from the channel.")


def main():
    # Add a handler for new messages in the channel
    dispatcher.add_handler(MessageHandler(Filters.chat_type.channel, send_new_channel_messages))
    # Add a handler for the /start command
    dispatcher.add_handler(CommandHandler("start", start))

    updater.start_polling()
    updater.idle()


if __name__ == '__main__':
    main()

2023-07-21 13:10:36,369 - apscheduler.scheduler - INFO - Scheduler started
2023-07-21 13:10:56,730 - telegram.ext.dispatcher - ERROR - No error handlers are registered, logging exception.
Traceback (most recent call last):
  File "/home/tonyalpha/.local/lib/python3.8/site-packages/telegram/ext/dispatcher.py", line 557, in process_update
    handler.handle_update(update, self, check, context)
  File "/home/tonyalpha/.local/lib/python3.8/site-packages/telegram/ext/handler.py", line 199, in handle_update
    return self.callback(update, context)
  File "<ipython-input-1-971e397ffdcf>", line 22, in send_new_channel_messages
    channel_updates = bot.get_chat_updates(channel_id)
AttributeError: 'Bot' object has no attribute 'get_chat_updates'
2023-07-21 13:11:06,016 - telegram.ext.updater - INFO - Received signal 2 (SIGINT), stopping...
2023-07-21 13:11:06,019 - apscheduler.scheduler - INFO - Scheduler has been shut down


In [192]:
a[2]['text'].

SyntaxError: invalid syntax (<ipython-input-192-20c5fbbb96dc>, line 1)

# Config upload

In [2]:
import configparser
import json

from telethon import TelegramClient
from telethon.errors import SessionPasswordNeededError

In [3]:
# Reading Configs
config = configparser.ConfigParser()
config.read("config.ini")

# Setting configuration values
api_id = config['Telegram']['api_id']
api_hash = config['Telegram']['api_hash']
api_token = config['Telegram']['api_token']

api_hash = str(api_hash)

phone = config['Telegram']['phone']
username = config['Telegram']['username']
channel_link = config['Telegram']['channel_link'] 

db_name = config['database']['db_name']
db_user = config['database']['db_user']
db_password = config['database']['db_password'] 
db_host = config['database']['db_host'] 


In [258]:
if not client.is_user_authorized():
    client.send_code_request(phone)
    try:
        client.sign_in(phone, input('Enter the code: '))
    except SessionPasswordNeededError:
        client.sign_in(password=input('Password: '))

me = client.get_me()
print(me)

<coroutine object UserMethods.get_me at 0x7f86fd106540>


<ipython-input-258-bc2b6df2013c>:1: RuntimeWarning: coroutine 'UserMethods.is_user_authorized' was never awaited
  if not client.is_user_authorized():
<ipython-input-258-bc2b6df2013c>:8: RuntimeWarning: coroutine 'UserMethods.get_me' was never awaited
  me = client.get_me()


In [3]:
chat_id = 'my life'

from telethon import TelegramClient
from telethon.tl.functions.messages import GetHistoryRequest
import telethon.sync

client = TelegramClient(None,
                    api_id,
                    api_hash)

assert client.connect()
if not client.is_user_authorized():
    client.send_code_request(phone_number)
    me = client.sign_in(phone_number, input('Enter code: '))

channel_username='my life' # your channel
channel_entity=client.get_entity(channel_username)
posts = client(GetHistoryRequest(
    peer=channel_entity,
    limit=100,
    offset_date=None,
    offset_id=0,
    max_id=0,
    min_id=0,
    add_offset=0,
    hash=0))
# messages stored in `posts.messages`

<ipython-input-3-adc2bb114a1f>:11: RuntimeWarning: coroutine 'TelegramBaseClient.connect' was never awaited
  assert client.connect()
<ipython-input-3-adc2bb114a1f>:12: RuntimeWarning: coroutine 'UserMethods.is_user_authorized' was never awaited
  if not client.is_user_authorized():


In [21]:
from telethon import TelegramClient, events, sync

client = TelegramClient(None, api_id, api_hash)
client.start()
channel_username = 'my life'
for message in client.get_messages(channel_username, limit=10):
    print(message.message)

<ipython-input-21-e72b7ee019a6>:4: RuntimeWarning: coroutine 'AuthMethods._start' was never awaited
  client.start()
<ipython-input-21-e72b7ee019a6>:6: RuntimeWarning: coroutine 'MessageMethods.get_messages' was never awaited
  for message in client.get_messages(channel_username, limit=10):


TypeError: 'coroutine' object is not iterable

In [24]:
from telethon.sync import TelegramClient



# We have to manually call "start" if we want an explicit bot token
bot = TelegramClient('bot', api_id, api_hash).start(bot_token=api_token)

# But then we can use the client instance as usual

In [25]:
from telethon.tl.functions.channels import GetParticipantsRequest
from telethon.tl.types import ChannelParticipantsSearch
from telethon.tl.types import (
PeerChannel
)

In [26]:

user_input_channel = input("enter entity(telegram URL or entity id):")

if user_input_channel.isdigit():
    entity = PeerChannel(int(user_input_channel))
else:
    entity = user_input_channel

my_channel = client.get_entity(entity)

enter entity(telegram URL or entity id):https://web.telegram.org/k/#-1810810362


In [28]:

offset = 0
limit = 100
all_participants = []

while True:
    participants = client(GetParticipantsRequest(
        my_channel, ChannelParticipantsSearch(''), offset, limit,
        hash=0
    ))
    if not participants.users:
        break
    all_participants.extend(participants.users)
    offset += len(participants.users)

AttributeError: 'coroutine' object has no attribute 'users'

In [29]:
from telethon.tl.functions.messages import (GetHistoryRequest)
from telethon.tl.types import (
PeerChannel
)

In [30]:

offset_id = 0
limit = 100
all_messages = []
total_messages = 0
total_count_limit = 0

while True:
    print("Current Offset ID is:", offset_id, "; Total Messages:", total_messages)
    history = client(GetHistoryRequest(
        peer=my_channel,
        offset_id=offset_id,
        offset_date=None,
        add_offset=0,
        limit=limit,
        max_id=0,
        min_id=0,
        hash=0
    ))
    if not history.messages:
        break
    messages = history.messages
    for message in messages:
        all_messages.append(message.to_dict())
    offset_id = messages[len(messages) - 1].id
    total_messages = len(all_messages)
    if total_count_limit != 0 and total_messages >= total_count_limit:
        break

Current Offset ID is: 0 ; Total Messages: 0


AttributeError: 'coroutine' object has no attribute 'messages'

In [8]:
import asyncio
from telethon import TelegramClient
from telethon.tl import functions, types



bot = TelegramClient(None, api_id, api_hash)
bot.start(bot_token=api_token)


async def main():
    channel = await client.get_entity('my life')
    messages = await client.get_messages(channel, limit= None) #pass your own args

    #then if you want to get all the messages text
    for x in messages:
        print(x.text) #return message.text

        
bot.disconnect()


<ipython-input-8-ac3a12e7944d>:8: RuntimeWarning: coroutine 'AuthMethods._start' was never awaited
  bot.start(bot_token=api_token)


<Future pending cb=[shield.<locals>._outer_done_callback() at /usr/lib/python3.8/asyncio/tasks.py:902]>

In [124]:
datetime.date.today() - datetime.timedelta(days=2)

datetime.date(2023, 7, 20)

In [116]:
from telethon.tl.types import InputMessagesFilterPhotos
from telethon import TelegramClient, events
import nest_asyncio

nest_asyncio.apply()

import os.path

def get_posts(num_posts = 1, download_media = False, only_with_media = True, offset_id = 0, posts_list = []):
    client = TelegramClient(username, api_id, api_hash)
    posts_list = []
    async def main(num_posts, download_media, offset_id, posts_list):
        
        if num_posts < 100:
            message_limit = num_posts
        else:
            message_limit = 100
            
            
        last_post_date = None
        post = {}
        
        await client.start()

        try:
            entity = await client.get_entity(channel_link)

            while True:
                messages = await client.get_messages(entity, limit=message_limit, offset_id = offset_id)

                for message in messages:
                    
                    if only_with_media:
                        if not message.media:
                            continue
                    
                    if last_post_date == None:
                        last_post_date = message.date


                    if last_post_date != message.date: 
                        post['upload_date'] = last_post_date.strftime('%Y-%m-%d %H:%M:%S') if last_post_date else None

                        last_post_date = message.date

                        posts_list.append(post)
                        post = {}


                    if len(message.message) > 0:
                        post['text'] = post.get('text', '') + message.message 
                        
                        # post_id in messages set will be id of message with text and photo 
                        post['post_id'] = message.id


                    if message.media and download_media:

                        photo_id = message.media.photo.id
                        filename = f'image_{photo_id}.jpg'
                        path = f"./media/{filename}"

                        # some of this i should drop, but later 
                        post['photos_id_list'] = post.get('photos_id_list', []) + [photo_id]
                        post['photos_names_list'] = post.get('photos_names_list', []) + [filename]

                        #if file exist check 
                        if not os.path.isfile(path) and download_media:
                            await client.download_media(message, file=path)

                    post['id_list'] = post.get('id_list', []) + [message.id]
                    post['edit_date'] = message.edit_date.strftime('%Y-%m-%d %H:%M:%S') if message.edit_date else None


                offset_id = messages[len(messages) - 1].id

                if len(posts_list) >= num_posts:
                    break


        except Exception as e:
            print(f"Error: {e}")

        await client.disconnect()
        
        return posts_list

    if __name__ == "__main__":
        client.loop.run_until_complete(main(num_posts, download_media, offset_id, posts_list))
        
    return posts_list

In [123]:
get_posts(num_posts = 1)

[{'text': 'asdfsdf',
  'post_id': 700,
  'id_list': [700],
  'edit_date': None,
  'upload_date': '2023-07-24 14:39:42'}]

In [14]:
a[-1]['text']

'05.04.23\n\n👨\u200d👨\u200d👦\u200d👦 🤦🏼 Did not find team for HSE hackaton\n💁🏼\u200d♂️💬 ODS random coffee with Roman\n📚 DL: semianr\n\nDay rate: 5.5/10'

In [308]:
 a[0]['text'].split('\n')

['чпок',
 '',
 'How productive have you been?:6.999/10',
 'How interesting was the day?: 6/10',
 'How stressful was the day?: 2/10']

# Data parsing

In [138]:
import re 



def parse_text(dct):
    
    input_string = dct['text']
    print(input_string)
    
    date_pattern = r'\d{2}\.\d{2}\.\d{2}'
    day_pattern = r'\b(?:MONDAY|TUESDAY|WEDNESDAY|THURSDAY|FRIDAY|SATURDAY|SUNDAY)\b'
    
    date_match = re.search(date_pattern, input_string)
    day_match = re.search(day_pattern, input_string)
    
    date = date_match.group() if date_match else None
    day = day_match.group() if day_match else None
    
    parsed_date = date.replace('.', '-') if date or day else None
    
    dct['parsed_date'] = parsed_date
    
    if date_match:
        input_string = input_string.replace(date_match.group(), "") if parsed_date is not None else input_string
    if day_match:
        input_string = input_string.replace(day_match.group(), "") if parsed_date is not None else input_string
    
    regex_list = [r'How productive have you been\?:\s*(\d+\.?\d*)/10', r'How interesting was the day\?:\s*(\d+\.?\d*)/10', r'How stressful was the day\?:\s*(\d+\.?\d*)/10']
    names_list = ['productivity_score', 'interest_score', 'stress_score']
    
    
    for regex, name in zip(regex_list, names_list):
        match = re.search(regex, input_string)
        parsed_score = float(match.group(1)) if match else None
        
        dct[name] = parsed_score

        input_string = input_string.replace(match.group(), "") if parsed_score is not None else input_string
        
    
    dct['parsed_text'] = input_string.split('\n')
    
    print('Text parsed!')
    return dct
    
def parse_activities(dct):
    
    input_list = dct['parsed_text']
    
    result = []
    for item in input_list:
        match = re.match(r'^([\U0001F000-\U0001F9FF]+|[\U0001FA00-\U0001FA6F]+)?\s*(.*)$', item)
        if match:
            emoji = match.group(1)
            text = match.group(2).strip()
            if emoji or text:
                result.append((emoji, text))
                
    dct['parsed_activities'] = result
    
    print('Activities parsed!')
    return dct

    

In [ ]:
from transformers import pipeline
classifier = pipeline("text-classification", model="SamLowe/roberta-base-go_emotions", return_all_scores=True)
classifier("I love this!")

In [ ]:
print('hi!')

In [97]:
parse_activities(parse_text(a[30]))

24.06.23


{'id_list': [564, 563, 562, 561],
 'edit_date': None,
 'text': '24.06.23',
 'post_id': 561,
 'upload_date': '2023-06-28 15:43:13',
 'parsed_date': '24-06-23',
 'productivity_score': None,
 'interest_score': None,
 'stress_score': None,
 'parsed_text': [''],
 'parsed_activities': []}

In [151]:
a[30]

{'id_list': [564, 563, 562, 561],
 'edit_date': None,
 'text': '24.06.23',
 'post_id': 561,
 'upload_date': '2023-06-28 15:43:13'}

In [383]:
a[79]['text']

'24.05.23\n\n* IAD: control work \n\n* My project for resume \n* My project: life analyze\n\n* English speaking club \n\n* Optics: think about future....\n\nHow productive have you been?: */10\nHow interesting was the day?: */10\nHow stressful was the day?: */10\nPAI: *'

In [40]:
#!pip install emoji

12.01.23
16.02.23


# Upload to DB 

Connected to database successfully


In [44]:
# Получаем список всех пользователей
cursor.execute("SELECT * FROM pg_catalog.pg_tables")

In [84]:
import mysql.connector

from getpass import getpass
from mysql.connector import connect, Error


def db_connect():
    try:
        with connect(host=db_host, user=db_user, password=db_password,) as connection:
            use_db_query = f"USE {db_name};"
            create_posts_table_query = """
            CREATE TABLE IF NOT EXISTS posts(
                post_id INT,
                text TEXT,
                parsed_date DATE,
                upload_date DATETIME,
                edit_date DATETIME, 
                productivity_score FLOAT,
                interest_score FLOAT,
                stress_score FLOAT,
                PRIMARY KEY (post_id)
            );
            """
            
            create_posts_table_activities = """
            CREATE TABLE IF NOT EXISTS activities(
                activity_id INT AUTO_INCREMENT,
                post_id INT,
                activity_name TEXT,
                activity_emoji TEXT,
                activity_type TEXT,
                activity_emotion TEXT,
                
                FOREIGN KEY (post_id) REFERENCES posts(post_id),
                PRIMARY KEY (activity_id)
            );
            """
            
            
            with connection.cursor() as cursor:
                cursor.execute(use_db_query)
                cursor.execute(create_posts_table_query)
                cursor.execute(create_posts_table_activities)
                connection.commit()
    except Error as e:
        print(e)
        
def upload_post(dct):
    try:
        with connect(host=db_host, user=db_user, password=db_password,) as connection:
            use_db_query = f"USE {db_name};"
            check_if_exist = """SELECT EXISTS(SELECT 1 FROM posts WHERE post_id = %s LIMIT 1)"""
            
            upload_posts_table_query = f"""
            INSERT INTO posts (post_id, text, parsed_date, upload_date, 
            edit_date,productivity_score, interest_score, stress_score) 
            VALUES(%s, %s, %s, %s, %s, %s, %s, %s)
            """
            
            post_id = (dct.get('post_id', None),)
            
            insert_data = (dct.get('post_id', None), dct.get('text', None), 
                    dct.get('parsed_date', None), dct.get('upload_date', None), 
                    dct.get('edit_date', None),dct.get('productivity_score', None),
                    dct.get('interest_score', None),dct.get('stress_score', None))
            
            with connection.cursor() as cursor:
                cursor.execute(use_db_query)
                cursor.execute(check_if_exist, post_id)
                for el in cursor:
                    exist = el[0]
                
                if not exist:
                    cursor.execute(upload_posts_table_query, insert_data)
                    connection.commit()
                else:
                    print('ERROR: post already uploaded, if you need update it use update_post()')
    except Error as e:
        print(e)
        
        
def upload_activities(dct):
    try:
        with connect(host=db_host, user=db_user, password=db_password,) as connection:
            use_db_query = f"USE {db_name};"
            check_if_exist = """SELECT EXISTS(SELECT 1 FROM posts WHERE post_id = %s LIMIT 1)"""
            
            upload_posts_table_query = f"""
            INSERT INTO posts (post_id, text, parsed_date, upload_date, 
            edit_date,productivity_score, interest_score, stress_score) 
            VALUES(%s, %s, %s, %s, %s, %s, %s, %s)
            """
            
            post_id = (dct.get('post_id', None),)
            
            insert_data = (dct.get('post_id', None), dct.get('text', None), 
                    dct.get('parsed_date', None), dct.get('upload_date', None), 
                    dct.get('edit_date', None),dct.get('productivity_score', None),
                    dct.get('interest_score', None),dct.get('stress_score', None))
            
            with connection.cursor() as cursor:
                cursor.execute(use_db_query)
                cursor.execute(check_if_exist, post_id)
                for el in cursor:
                    exist = el[0]
                
                if not exist:
                    cursor.execute(upload_posts_table_query, insert_data)
                    connection.commit()
                else:
                    print('ERROR: post already uploaded, if you need update it use update_post()')
    except Error as e:
        print(e)
        
        
def update_post(dct):
    try:
        with connect(host=db_host, user=db_user, password=db_password,) as connection:
            use_db_query = f"USE {db_name};"
            update_posts_table_query = """
            UPDATE posts SET text = %s, parsed_date = %s,  
            upload_date = %s, edit_date = %s, productivity_score = %s,
            interest_score = %s, stress_score = %s WHERE post_id = %s
            """
            
            data = (dct.get('text', None), 
                    dct.get('parsed_date', None), dct.get('upload_date', None), 
                    dct.get('edit_date', None),dct.get('productivity_score', None),
                    dct.get('interest_score', None),dct.get('stress_score', None),
                   dct.get('post_id', None))

            with connection.cursor() as cursor:
                cursor.execute(use_db_query)
                cursor.execute(update_posts_table_query, data)
                connection.commit()
    except Error as e:
        print(e)

def db_show_tables():
    try:
        with connect(host=db_host, user=db_user, password=db_password,) as connection:
            use_db_query = f"USE {db_name};"
            show_tables = "SHOW TABLES;"
            with connection.cursor() as cursor:
                cursor.execute(use_db_query)
                cursor.execute(show_tables)
                for el in cursor.fetchall():
                    print(el)
    except Error as e:
        print(e)
        
def db_execute_query(query):
    try:
        with connect(host=db_host, user=db_user, password=db_password,) as connection:
            use_db_query = f"USE {db_name};"
            with connection.cursor() as cursor:
                cursor.execute(use_db_query)
                cursor.execute(query)
                for el in cursor:
                    print(el)
    except Error as e:
        print(e)

In [85]:
db_connect()

In [459]:
db_show_tables()

('posts',)


In [551]:
db_execute_query('SELECT EXISTS(SELECT * FROM posts WHERE post_id = 9 LIMIT 1)')

(0,)


In [80]:
db_execute_query('SELECT * FROM posts;')

# Main

In [152]:
def check_post(post):
    
    date = post.get('upload_date', None)
    
    if post.get('parsed_date', None) is None:
        print(f'I dont see date, can you add it to post uploaded at {date}?')
        return False

    if post.get('productivity_score', None) is None:
        print(f'I dont see productivity_score, can you add it to post uploaded at {date}?')
        return False
        
    if post.get('interest_score', None) is None:
        print(f'I dont see interest_score, can you add it to post uploaded at {date}?')
        return False
        
    if post.get('productivity_score', None) is None:
        print(f'I dont see productivity_score, can you add it to post uploaded at {date}?')
        return False
        
    if post.get('stress_score', None) is None:
        print(f'I dont see stress_score, can you add it to postuploaded at {date}?')
        return False
    print('Post correct!')
    return True

def check_activities(post):
    
    date = post.get('upload_date', None)
    
    if post.get('parsed_activities', None) is None or len(post.get('parsed_activities', None)) < 0:
        print(f'I dont see activities, can you add some to post uploaded at {date}?')
        return False
    return True


In [153]:
posts = get_posts(num_posts = 1)

In [149]:
posts

[{'text': '05.04.23\nasdfsdf\n\nHow productive have you been?: 0/10\nHow interesting was the day?: 9/10\nHow stressful was the day?: 0/10',
  'post_id': 700,
  'id_list': [700],
  'edit_date': '2023-07-24 15:57:47',
  'upload_date': '2023-07-24 14:39:42'}]

In [150]:


posts = get_posts(num_posts = 2)

for post in posts:
    post = parse_text(post)
    post = parse_activities(post)
    if check_post(post):
        upload_post(post)
    
    if check_activities(post):
        upload_activities(post)


05.04.23
asdfsdf

How productive have you been?: 0/10
How interesting was the day?: 9/10
How stressful was the day?: 0/10
Text parsed!
Activities parsed!
Post correct!
ERROR: post already uploaded, if you need update it use update_post()
чпок

How productive have you been?:6.999/10
How interesting was the day?: 6/10
How stressful was the day?: 2/10
Text parsed!
Activities parsed!
I dont see date, can you add it to post uploaded at 2023-07-23 16:22:13?


In [1]:
print('hi')

hi


In [8]:
from transformers import pipeline

classifier = pipeline("text-classification", model="j-hartmann/emotion-english-distilroberta-base", top_k=1)
classifier("hello!")

[[{'label': 'neutral', 'score': 0.44008079171180725}]]

In [9]:
from transformers import pipeline

classifier = pipeline("text-classification", model="SamLowe/roberta-base-go_emotions", top_k=1)
classifier("hello!")

KeyboardInterrupt: 

# Sync with DB

# Etc 